In [0]:
import requests
from bs4 import BeautifulSoup

# filter crunch reports which are not news about FB
def filter_urls_in_file(filename):
  non_report_urls = []
  for url in open(filename):
    if "crunch-report" not in url: 
      non_report_urls.append(url)
      
  #update file
  f = open(filename, "w")
  for url in non_report_urls:
    f.write(url)   

def parse_url_file(filename):
  urls = []
  for url in open(filename):
    if "crunch-report" not in url: 
      urls.append(url.rstrip('\n'))
      
  return parse_all_urls(urls)

def parse_all_urls(urls):
  contents = []
  for url in urls:
    contents.append(parse_url(url))
  return contents

#format : tilte + "\" + content
def parse_url(url):
  # Make the request to a url
  r = requests.get(url)
  
  # Create soup from content of request
  c = r.content
  
  soup = BeautifulSoup(c)
  
  # Find the element on the webpage
  main_title = soup.find('h1', attrs = {'class': 'article__title'}) 
  #main_title

  main_content = soup.find('div' , attrs = {'class': 'article-content'})
  #main_content
  
  # Init
  title = ""
  content = ""
  
  # Parse title
  title = main_title.text
  
  # Extract content
  paragraphs = main_content.find_all('p')
  
  for p in paragraphs:
    content += (p.text + " ")
    
  return (title + "\n" + content) 
  

In [0]:
filename = "fb2017"

# Run for once
filter_urls_in_file(filename + ".txt")

In [0]:
# Parse file
contents = parse_url_file(filename + ".txt")
# print(contents)

In [0]:
# # Print all results for debuggin
# cnt = 1
# for content in contents:
#   print(cnt, content, '\n')
#   cnt += 1

In [0]:
# Write to file
f = open(filename + "_contents.txt", "w")
for content in contents:
  f.write(content + '\n')

## Testing with the sentiment part

In [6]:
import nltk

from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


True

In [32]:
# sentences = []
score_list = []
articles = contents

sid = SentimentIntensityAnalyzer()

for i in range(len(articles)):
  sentences = []
  lines_list = tokenize.sent_tokenize(articles[i])
#   print(lines_list)
  sentences.extend(lines_list)
#   print(len(sentences))
  
  scores = {}
  scores['pos'] = 0
  scores['neg'] = 0
  scores['neu'] = 0
  scores['compound'] = 0
  
#   print(sentences)
  
  for sentence in sentences:
#     print(sentence)
#     print(ss)
    ss = sid.polarity_scores(sentence)
  
    scores['pos'] += ss['pos']
    scores['neg'] += ss['neg']
    scores['neu'] += ss['neu']
    scores['compound'] += ss['compound']
  
  for k in sorted(scores):
    scores[k] /= len(sentences)
    
  print(i, scores)
  score_list.append(scores)

0 {'pos': 0.11978571428571429, 'neg': 0.005821428571428572, 'neu': 0.8743928571428571, 'compound': 0.44807857142857144}
1 {'pos': 0.1859, 'neg': 0.035800000000000005, 'neu': 0.7782, 'compound': 0.29922}
2 {'pos': 0.13123809523809524, 'neg': 0.0, 'neu': 0.8687619047619047, 'compound': 0.4448857142857142}
3 {'pos': 0.09383333333333332, 'neg': 0.0805, 'neu': 0.8256666666666667, 'compound': 0.030374999999999985}
4 {'pos': 0.08547619047619048, 'neg': 0.03480952380952381, 'neu': 0.8797142857142857, 'compound': 0.1847571428571429}
5 {'pos': 0.08025, 'neg': 0.0, 'neu': 0.91975, 'compound': 0.25295}
6 {'pos': 0.09514285714285715, 'neg': 0.14492857142857143, 'neu': 0.7597857142857142, 'compound': -0.19415}
7 {'pos': 0.07584210526315789, 'neg': 0.02589473684210526, 'neu': 0.793, 'compound': 0.15895263157894735}
8 {'pos': 0.058666666666666666, 'neg': 0.041666666666666664, 'neu': 0.8997, 'compound': 0.05301333333333332}
9 {'pos': 0.05711764705882353, 'neg': 0.036470588235294116, 'neu': 0.9064117647